<a href="https://colab.research.google.com/github/yalopez84/GAN_study/blob/master/GAN_14_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [153]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [154]:
class Discriminator(nn.Module):
    def __init__(self, img_dim):
        super().__init__()
        self.disc=nn.Sequential(
            nn.Linear(img_dim,128),
            nn.LeakyReLU(0.1),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        return self.disc(x)

In [155]:
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen=nn.Sequential(
            nn.Linear(z_dim,256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim),
            nn.Tanh()
        )
    def forward(self,x):
        return self.gen(x)

In [156]:
#hyperparameters etc.
device="cuda" if torch.cuda.is_available() else "cpu"
lr=3e-4
z_dim=64
image_dim=28 * 28 *1 # 784
batch_size=32
num_epochs=50
disc= Discriminator(image_dim).to(device)
gen=Generator(z_dim,image_dim).to(device)
fixed_noise=torch.randn((batch_size,z_dim)).to(device)
transforms=transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
dataset=datasets.MNIST(root="dataset/",transform=transforms, download=True)
loader=DataLoader(dataset,batch_size=batch_size,shuffle=True)
opt_disc=optim.Adam(disc.parameters(),lr=lr)
opt_gen=optim.Adam(gen.parameters(),lr=lr)
criterion=nn.BCELoss()
writer_fake= SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real= SummaryWriter(f"runs/GAN_MNIST/real")
step=0
for epoch in range(num_epochs):
    for batch_idx, (real,_) in enumerate (loader):
        real=real.view(-1,784).to(device)
        batch_size=real.shape[0]

        #train discriminator max log (D(real)) + log(1-D(G(z)))
        noise =torch.randn(batch_size,z_dim).to(device)
        fake=gen(noise)
        disc_real=disc(real).view(-1)
        lossD_real=criterion(disc_real, torch.ones_like(disc_real))
        disc_fake=disc(fake.detach()).view(-1)
        lossD_fake=criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD=(lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        #train generator min log log(1-D(G(z))) ->max log(D(G(z)))
        output=disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx ==0:
            print(f"Epoch [{epoch}/{num_epochs}] Loss D: {lossD: .4f}, Loss G: {lossG: 4f}")

            with torch.no_grad():
                fake=gen(fixed_noise). reshape(-1, 1, 28, 28)
                data=real.reshape(-1, 1, 28, 28)
                img_grid_fake=torchvision.utils.make_grid(fake,normalize=True)
                img_grid_real=torchvision.utils.make_grid(data, normalize=True)
                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist real Images", img_grid_real, global_step=step
                )
                step+=1


Epoch [0/50] Loss D:  0.7352, Loss G:  0.687961
Epoch [1/50] Loss D:  0.1882, Loss G:  2.076466
Epoch [2/50] Loss D:  0.0593, Loss G:  3.109229
Epoch [3/50] Loss D:  0.0779, Loss G:  3.842041
Epoch [4/50] Loss D:  0.0883, Loss G:  3.785535
Epoch [5/50] Loss D:  0.1266, Loss G:  3.814486
Epoch [6/50] Loss D:  0.0207, Loss G:  4.561933
Epoch [7/50] Loss D:  0.0369, Loss G:  4.755553
Epoch [8/50] Loss D:  0.0169, Loss G:  4.892198
Epoch [9/50] Loss D:  0.0144, Loss G:  5.677328
Epoch [10/50] Loss D:  0.0659, Loss G:  5.412946
Epoch [11/50] Loss D:  0.0126, Loss G:  5.646009


KeyboardInterrupt: ignored

In [ ]:
model=Discriminator()
model